'''
Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JoengYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.
'''

# Cheat Your Decision-Making Model: Manipulating Loan Approval Data with Causal Inference

## Introduction

In this tutorial, we explore how to manipulate decision-making models for loan approval prediction. Using a Cooperative Network (CCNet), we investigate the impact of subtle manipulations in non-fraudulent data to test and expose the vulnerabilities of machine learning models. This demonstration not only highlights the strengths and weaknesses of current fraud detection systems but also outlines the ethical considerations and potential risks associated with data manipulation.

## Tutorial Goals
This tutorial offers guidance on applying data manipulation to machine learning models for predicting loan approval outcomes and evaluating the impacts. We will address the following:

1. **Training a Binary Classifier**: We'll train a model using actual loan data to differentiate between approved and denied loan applications.
2. **Generating Manipulated Data with CCNet**: We'll employ the Explainer, Reasoner, and Producer networks within CCNet to create data that mimics genuine loan applications but is engineered to mislead the classifier by altering decision label factors.
3. **Evaluating Classifier Performance**: We'll test the classifier on both the original and the manipulated datasets to assess the model’s robustness. The classifier should perform well on both the original test set and the CCNet-recreated test set, demonstrating that CCNet has effectively learned how the classifier maps decisions.
4. **Comparative Dataset Analysis**:
   - Testset A: Real Test Data - Actual loan approval data used as a benchmark to evaluate the classifier’s performance.
   - Testset B: CCNet-generated data with random labels but maintaining the same frequency distribution as the original. The classifier's performance on this dataset should mirror that of Testset A, indicating that CCNet comprehends the classifier's decision-making process.
   - Testset C: CCNet-generated data with original labels. While Testset B proves that CCNet can generate a genuine data distribution for any scenario of decision label, most of the data's information resides in the explanation vector, making it easy to manipulate by changing the decision factor of the data. In short, the decision-making process can be easily manipulated through causal inference.
5. **Ethical Considerations**: We will discuss the ethical implications of manipulating data in machine learning, emphasizing the necessity for robustness in model training and the risks of misuse in critical applications like loan approval.

In [1]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

import warnings
warnings.filterwarnings("ignore")

In [2]:
# https://www.kaggle.com/datasets/architsharma01/loan-approval-prediction-dataset

import pandas as pd 

dataroot = path_append + "../data/LoanApprovalPredictionDataset/loan_approval_dataset.csv"
df = pd.read_csv(dataroot)
df.columns = df.columns.str.replace(' ', '')
df

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved


In [3]:
print('Approved', round(df['loan_status'].value_counts()[0] / len(df) *100,2), '%of the dataset')
print('Rejected', round(df['loan_status'].value_counts()[1] / len(df) *100,2), '%of the dataset')

Approved 62.22 %of the dataset
Rejected 37.78 %of the dataset


In [5]:
import torch
from sklearn.preprocessing import RobustScaler
from torch.utils.data import Dataset

one_hot_columns = ['education', 'self_employed']

df = pd.get_dummies(df, columns=one_hot_columns)

one_hot_columns = ['education_ Graduate', 'education_ Not Graduate', 'self_employed_ No', 'self_employed_ Yes']
df[one_hot_columns] = df[one_hot_columns].astype(int)

scale_cols = ['loan_id', 'no_of_dependents', 'income_annum', 'loan_amount',
       'loan_term', 'cibil_score', 'residential_assets_value',
       'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value']

# Initialize scalers
sc = RobustScaler()

# Scale all columns except the last one (which is the class column)
df[scale_cols] = sc.fit_transform(df[scale_cols])

# Map the target column to 0 and 1
df['loan_status'] = df['loan_status'].map({' Approved': 1, ' Rejected': 0})

# Drop the loan_id column
df.drop('loan_id', axis=1, inplace=True)

In [6]:
# delete the original loan_status column to change its position
loan_status = df.pop('loan_status')

df['loan_status'] = loan_status

In [7]:
df.head()

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,education_ Graduate,education_ Not Graduate,self_employed_ No,self_employed_ Yes,loan_status
0,-0.333333,0.937500,1.115942,0.2,0.603390,-0.351648,2.206349,0.570423,0.708333,1,0,1,0,1
1,-1.000000,-0.208333,-0.166667,-0.2,-0.620339,-0.318681,-0.238095,-0.408451,-0.270833,0,1,0,1,0
2,0.000000,0.833333,1.101449,1.0,-0.318644,0.164835,0.126984,1.316901,1.708333,1,0,1,0,0
3,0.000000,0.645833,1.173913,-0.2,-0.450847,1.384615,-0.063492,0.612676,0.687500,1,0,1,0,0
4,0.666667,0.979167,0.702899,1.0,-0.738983,0.747253,0.714286,1.042254,0.083333,0,1,0,1,0


In [8]:
# Defining the labeled and unlabeled dataset classes
class LabeledDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype=torch.float32)
        label = torch.tensor(self.y[index], dtype=torch.long)
        # label = torch.nn.functional.one_hot(label, num_classes=2).float().squeeze(0)
        return vals, label

#### Dataset Splitting for Training and Testing

The original dataset is split into training and testing parts to evaluate the model's performance accurately. This step is crucial for validating the effectiveness of the training on unseen data.


In [9]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets for model evaluation
df_train, df_test = train_test_split(df, test_size=0.7, shuffle=True, random_state=42)
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:, -1:].values

# Labeled datasets for supervised learning tasks
trainset = LabeledDataset(X_train, y_train)  # Corrected to include training data
testset = LabeledDataset(X_test, y_test)     # Test set with proper labels

# Printing the shapes of the datasets for verification
print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

Labeled Trainset Shape: 1280, 13
Labeled Testset Shape: 2989, 13


In [10]:
# Calculate the number of features and classes
n_features = trainset[0][0].shape[0]
n_classes = trainset[0][1].shape[0]

print(n_features, n_classes)

13 1


#### Initial Setup and Model Configuration

This section initializes the environment by setting a fixed random seed to ensure reproducibility of results. It imports necessary configurations and initializes model parameters with specific configurations. The model specified here is set to have no core model but uses a 'tabnet' encoder model for data processing, which is particularly tailored for structured or tabular data.


In [11]:
# Set a fixed random seed for reproducibility of experiments
from nn.utils.init import set_random_seed
set_random_seed(0)

# Importing configuration setups for ML parameters and data
import torch
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub

# Configuration for the data handling, defining dataset specifics and the task type
data_config = DataConfig(dataset_name='LoanApproval', task_type='binary_classification', obs_shape=[n_features], label_size=n_classes, explain_size=4)

# Initializing ML parameters without a core model and setting the encoder model to 'tabnet' with specific configurations
ml_params = MLParameters(ccnet_network='tabnet', encoder_network='none')

# Setting training parameters and device configuration
ml_params.training.num_epoch = 50
ml_params.model.ccnet_config.num_layers = 3
ml_params.algorithm.error_function = 'mse'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Create a TrainerHub instance to manage training and data processing
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False, print_interval=200)


In [12]:
# Generation loss should close to 0.0
load_model = False
if load_model:
    trainer_hub.load_trainer(ccnet_network = True)
else:
    trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

...

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

Iterations:   0%|          | 0/20 [00:00<?, ?it/s]

In [13]:
# DataLoader for processing training data in larger batches without shuffling
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=256, shuffle=False, drop_last=False)

# CCNet setup for generating synthetic manipulated data
ccnet = trainer_hub.ccnet
manipulated_data = None
random_labels = None
original_labels = None
probability_positive = 0.6222 # 62.2% of the data is positive

# Generate synthetic data to enhance the diversity of the training dataset
for data, labels in test_loader:
    data = data.to(device)
    labels = labels.to(device)
    
    # Create random labels with 62.2% positive (1) and 37.8% negative (0) labels
    random_label = (torch.rand(labels.size(0)).to(device) < probability_positive).float()
    random_label = torch.nn.functional.one_hot(random_label.to(torch.int64), num_classes=2).float()

    # Use CCNet to explain the original data and generate synthetic counterparts
    explanations = ccnet.explain(data)
    synthetic_data = ccnet.produce(random_label, explanations)
    
    # Detach synthetic data and labels from GPU for further processing
    synthetic_data = synthetic_data.detach().cpu()
    random_label = random_label.detach().cpu().argmax(dim=-1).unsqueeze(-1)
    labels = labels.detach().cpu()

    # Accumulate the generated data for analysis and training
    if manipulated_data is None:
        manipulated_data = synthetic_data
        random_labels = random_label
        original_labels = labels
    else:
        manipulated_data = torch.cat([manipulated_data, synthetic_data], dim=0)
        random_labels = torch.cat([random_labels, random_label], dim=0)
        original_labels = torch.cat([original_labels, labels], dim=0)

# Output the shapes of the datasets for verification
print(f"Manipulated Data Shape: {manipulated_data.shape}")
print(f"Reversed Labels Shape: {random_labels.shape}")
print(f"Original Labels Shape: {original_labels.shape}")

# Create datasets for both synthetic and original label scenarios
random_label_testset = LabeledDataset(manipulated_data.numpy(), random_labels.numpy())
original_label_testset = LabeledDataset(manipulated_data.numpy(), original_labels.numpy())


Manipulated Data Shape: torch.Size([2989, 13])
Reversed Labels Shape: torch.Size([2989, 1])
Original Labels Shape: torch.Size([2989, 1])


#### Training Supervised Models

This section outlines the process of training supervised learning models using both original and synthetic datasets. The `train_supervised_model` function is designed to iterate through the dataset, perform forward passes, compute loss, and update model weights using backpropagation.


In [14]:
def train_supervised_model(model, dataset, num_epoch=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Ensure reproducibility by resetting the random seed
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    # Training loop
    for epoch in range(num_epoch):  # Train for 2 epochs as an example
        for i, (data, label) in enumerate(trainloader):
            data = data.to(device).clone().detach()
            label = label.to(device).float()
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()


In [15]:

class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=256):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)
    
# Initialize and train a model on the recreated dataset
decision_making_model = MLP(input_size=n_features, output_size=n_classes).to(device)
train_supervised_model(decision_making_model, trainset)

#### Evaluating Model Performance


In [16]:
from sklearn.metrics import f1_score

def get_f1_score(model, input_testset, device, batch_size=64):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    # DataLoader for testing
    test_loader = torch.utils.data.DataLoader(input_testset, batch_size=batch_size, shuffle=False)

    # No gradient computation needed during inference
    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            # Process output for binary classification
            predicted = (output.squeeze() > 0.5).long()
            y_true.extend(label.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute and return the F1 score
    score = f1_score(y_true, y_pred, average='binary')
    return score

# Calculate F1 scores for both models
fraud_detection_f1_score = get_f1_score(decision_making_model, testset, device)
random_testset_f1_score = get_f1_score(decision_making_model, random_label_testset, device)
manipulated_testset_f1_score = get_f1_score(decision_making_model, original_label_testset, device)

# Output the results
print("F1 score of the supervised learning model tested on the original data: ", fraud_detection_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with random label: ", random_testset_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with original label: ", manipulated_testset_f1_score)

F1 score of the supervised learning model tested on the original data:  0.9380873410724158
F1 score of the supervised learning model tested on the manipulated data with random label:  0.9989229940764675
F1 score of the supervised learning model tested on the manipulated data with original label:  0.6257104194857915
